# Fix wave or ripple in error

Our data has a ripple pattern in the error through time.
This notebooks investigates it an tries to come up with a good explanation so that we can move forward.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import dask
import dask.array as da
import dask.dataframe as dd
import dask.distributed
import dask_jobqueue
import numpy as np
import os
import pandas as pd
import pathlib
import plotly.express as px

In [ ]:
DATA_DIR = pathlib.Path(os.getenv('DATA_DIR'))
INPUT_DATASET = DATA_DIR / 'interpolated/2021-12-20-test/'

In [ ]:
input_path = pathlib.Path(INPUT_DATASET)

In [ ]:
sample_path = next(iter(input_path.glob('*.parquet')))

In [ ]:
sample = pd.read_parquet(sample_path)

In [ ]:
columns = set(sample.columns)
columns -= set(['gdps_hpbl'])

In [ ]:
cluster = dask_jobqueue.SLURMCluster(
    env_extra=['source ~/.bash_profile','conda activate smc01'],
    name='smc01-dask',
)

In [ ]:
cluster.scale(jobs=8)

In [ ]:
client = dask.distributed.Client(cluster)

In [ ]:
client

In [ ]:
df = dd.read_parquet(list(iter(input_path.glob('*.parquet'))), columns=columns)


In [ ]:
df.head()

In [ ]:
df = client.persist(df)

In [ ]:
df = df.reset_index()
df['step_hour'] = df['step'] / 3600
df['error_2t'] = df['obs_2t'] - df['gdps_2t']
df['squared_error_2t'] = (df['gdps_2t'] - df['obs_2t']) ** 2
df['rmse_2t'] = da.sqrt(df['squared_error_2t'])
df['mabs_2t'] = np.abs(df['error_2t'])
df['forecast_month'] = df['date'].dt.month
df['forecast_hour'] = df['date'].dt.hour

In [ ]:
df = df.set_index('date')

In [ ]:
df['step_td'] = dd.to_timedelta(df['step'], unit='S')


In [ ]:
df['valid'] = df.index + df['step_td']


## Error by valid time

In [ ]:
df['valid_hour'] = df['valid'].dt.hour

In [ ]:
by_valid_hour = df.groupby(['valid_hour', 'forecast_hour', 'step_hour']).agg({'squared_error_2t': 'mean', 'station': 'count'}).compute()

In [ ]:
by_valid_hour = by_valid_hour.reset_index()


In [ ]:
by_valid_hour

In [ ]:
px.box(data_frame=by_valid_hour, x='valid_hour', y='squared_error_2t', color='forecast_hour')

In [ ]:
px.box(data_frame=by_valid_hour, x='step_hour', y='squared_error_2t', color='forecast_hour')

In [ ]:
px.box(data_frame=by_valid_hour, x='step_hour', y='station', color='forecast_hour')

In [ ]:
by_valid_hour.head()

In [ ]:
px.box(data_frame=by_valid_hour, x='valid_hour', y='station')

I bet some stations close at night and this is why we have a selection bias.

In [ ]:
cyul = df[df['station'] == 'CYUL'].compute()

In [ ]:
px.box(data_frame=cyul, x='step_hour', y='squared_error_2t')

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df = df.reset_index()

In [ ]:
df['forecast_date'] = df['date'].dt.date

In [ ]:
daily_station_counts = df.groupby(['station', 'forecast_date']).count()

In [ ]:
daily_station_counts.groupby('station').mean().head()

In [ ]:
counts = daily_station_counts.compute()

In [ ]:
px.histogram(data_frame=counts, x='date', log_y=True)

In [ ]:
len(counts[counts['date'] > 150])

In [ ]:
med = counts.groupby('station').median()

In [ ]:
px.histogram(data_frame=med, x='date', log_y=True)

In [ ]:
df['valid_date'] = df['valid'].dt.date

In [ ]:
valid_counts = df.groupby(['valid_date', 'station']).count().compute()

In [ ]:
by_station = valid_counts.groupby('station').median()

In [ ]:
px.histogram(data_frame=by_station, x='latitude')

In [ ]:
df['valid_day_of_week'] = df['valid'].dt.dayofweek

In [ ]:
valid_counts = df.groupby(['valid_day_of_week', 'station']).count().compute()

In [ ]:
smallest_day = valid_counts.groupby('station').count()

In [ ]:
valid_counts

In [ ]:
px.histogram(valid_counts, x='date')

In [ ]:
by_valid_hour = df.groupby('valid_hour').mean().compute()

In [ ]:
px.scatter(by_valid_hour, x=by_valid_hour.index, y='step_hour')

In [ ]:
hourly_counts = df.groupby(['station', 'valid_hour']).count().compute()

In [ ]:
hourly_counts = hourly_counts.reset_index()

In [ ]:
nparray = hourly_counts.pivot(index='station', columns='valid_hour', values='index').to_numpy()

In [ ]:
nparray.shape

In [ ]:
px.imshow(nparray, width=600, height=4000, aspect='auto')

In [ ]:
hourly_counts.groupby('station')